# Structured Output with Function Calling

As developers, we often want our models to return structured output (as opposed to raw text), so they can interface with other systems. There's a range of [interesting](https://x.com/goodside/status/1657396491676164096?s=20) ways of doing this. This notebook will specifically look at the **function calling** approach, but we'll briefly go over alternatives at the end.



**First off, why use function calling for JSON? What do function calls have to do with JSON output?**


### Motivation

What does calling functions have to do with JSON output?

Strictly speaking, it doesn't! The key idea is that both involve _structure_ (keys, values, and types), so we can leverage the **inherent structure in the typed arguments of a function** to model the **JSON we want to output**.

Let's look at an example:

Say we want to extract the `name`, `date`, `event_type`, and `attendees` of an event given some natural language input.

Input:
```
Next tuesday Jennifer is hosting a house warming at her place, and so far she's invited Steven and Julian. Alex told me he's going with Jessica, but Samantha can't make it.
```

Desired Output:
```javascript
{
    "name": "Jennifer's Housewarming",
    "date": "Next Tuesday", // (to be computed later)
    "event_type": "SOCIAL",
    "attendees": ["Steven", "Julian", "Jessica"]
}
```

We could hypothetically caputre this structre in a function's argument by defining it like so:

```python
def define_event(name: str, date: str, attendees: List[str]):
    pass
```

We don't actually have to define this function in python – just define it's interface to pass to our model in the `functions` param! We can optionally use descriptions to help the model understand what an argument is meant to represent.

In [41]:
functions = [
    {
        "name": "define_event",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                },
                "date": {
                    "type": "string",
                    "description": "Can be either absolute or relative e.g. Next Week.",
                },
                "event_type": {"type": "string", "enum": ["SOCIAL", "WORK", "OTHER"]},
                "attendees": {"type": "array", "items": {"type": "string"}},
            },
        },
    }
]

Next, we can make the call to OpenAI by providing the function in the `functions` param. We'll also set the `function_call` param to `{"name":"define_event"}` to force the model to call that function.

In [42]:
import openai

messages = [
    {
        "role": "system",
        "content": "Extract the event details.",
    },
    {
        "role": "user",
        "content": "Next tuesday Jennifer is hosting a house warming at her place, and so far she's invited Steven and Julian. Alex told me he's going with Jessica, but Samantha can't make it.",
    },
]

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages,
    functions=functions,
    function_call={"name": "define_event"},
)
response

<OpenAIObject chat.completion id=chatcmpl-8HMQCDtgR9NDjRL2G1FRFAV8CLrfe at 0x13a6facc0> JSON: {
  "id": "chatcmpl-8HMQCDtgR9NDjRL2G1FRFAV8CLrfe",
  "object": "chat.completion",
  "created": 1699148456,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "define_event",
          "arguments": "{\n  \"name\": \"Jennifer's House Warming\",\n  \"date\": \"Next Tuesday\",\n  \"event_type\": \"SOCIAL\",\n  \"attendees\": [\"Steven\", \"Julian\", \"Alex\", \"Jessica\"]\n}"
        }
      },
      "finish_reason": "stop",
      "internal_metrics": [
        {
          "cached_prompt_tokens": 0,
          "total_accepted_tokens": 0,
          "total_batched_tokens": 173,
          "total_predicted_tokens": 0,
          "total_rejected_tokens": 0,
          "total_tokens_in_completion": 174,
          "cached_embeddings_bytes": 0,
          "cached_embeddings_n": 0,

Finally, we extract the function call provided by the model. Note the arguments are JSON encoded, so we'll need to decode those as well.

In [44]:
import json

event_details = json.loads(response.choices[0].message.function_call.arguments)
event_details

{'name': "Jennifer's House Warming",
 'date': 'Next Tuesday',
 'event_type': 'SOCIAL',
 'attendees': ['Steven', 'Julian', 'Alex', 'Jessica']}

Awesome! Now all together wrapped ina  

In [ ]:
import openai

def extract_event_details(user_input):
    messages = [
        {
            "role": "system",
            "content": "Extract the event details.",
        },
        {
            "role": "user",
            "content": user_input,
        },
    ]
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        functions=functions,
        function_call={"name":"define_event"},
    )
    return response

response = extract_event_details("Next tuesday Jennifer is hosting a house warming at her place, and so far she's invited Steven and Julian. Alex told me he's going with Jessica, but Samantha can't make it.")

In [ ]:
import json

event_details = json.loads(response)

In [ ]:
import openai, json

SAMPLE_JSON = {
    "date": "2021-10-10",
    "event_type": "social",
}

JSON_SCHEMA = {
    "type": "object",
    "properties": {
        "date": {
            "type": "string",
        },
        "event_type": {"type": "string", "enum": ["social", "work"]},
    },
}


def extract_json(input_text, json_schema):
    extract_function_name = "extract_json"
    functions = [{"name": extract_function_name, "parameters": json_schema}]

    messages = [
        {
            "role": "system",
            "content": "Extract the relevant fields by using the return_json function.",
        },
        {
            "role": "user",
            "content": input_text,
        },
    ]

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        functions=functions,
        function_call={"name":extract_function_name},
    )
    return json.loads(response.choices[0].message.function_call.arguments)


print(extract_json("The date is 2021-10-10 and the event type is social.", JSON_SCHEMA))

In [ ]:
import requests
import os

def maybe_json():
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
    }
    data = {
        "model": "gpt-3.5-turbo-1106",
        # "response_format": { "type": "json_object" },
        "max_tokens": 200,
        "messages": [
            {
                "role": "user",
                "content": "Tell me a two sentence story about JSON."
            }
        ]
    }
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=data)
    return response.json()

print(json.dumps(maybe_json(), indent=4))
